In [1]:
import sys
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

# --- SETUP MOCK ENVIRONMENT FOR LOCAL RUN ---
try:
    # พยายาม Import ของจริง (เผื่อรันบน Glue Docker หรือ Cloud)
    from awsglue.context import GlueContext
    from awsglue.job import Job
    from awsglue.utils import getResolvedOptions
    print("✅ Found AWS Glue Libraries (Running on AWS/Docker)")
except ImportError:
    # ถ้ารันบน Laptop ทั่วไป ให้สร้าง Mock Class ขึ้นมาแทน
    print("⚠️ AWS Glue Libraries NOT found. Using MOCK objects for Local Laptop.")
    
    # 1. Mock GlueContext
    class GlueContext:
        def __init__(self, sc):
            self.sc = sc
            # ใช้ Local SparkSession แทน Glue SparkSession
            self.spark_session = SparkSession.builder \
                .master("local[*]") \
                .appName("LocalGlueRun") \
                .getOrCreate()

    # 2. Mock Job Class
    class Job:
        def __init__(self, glue_context):
            pass
        def init(self, job_name, args):
            print(f"   -> [Mock] Job Initialized: {job_name}")
        def commit(self):
            print("   -> [Mock] Job Committed")

    # 3. Mock getResolvedOptions
    def getResolvedOptions(args, options):
        # คืนค่า Dictionary เปล่า หรือค่า Mock ตามที่ Key ที่ขอมา
        return {k: 'mock_value' for k in options}

⚠️ AWS Glue Libraries NOT found. Using MOCK objects for Local Laptop.


In [2]:
import time
import concurrent.futures
import pandas as pd
# ไม่ต้อง import awsglue ตรงนี้แล้ว เพราะ Cell บนจัดการให้แล้ว

# ==========================================
# 1. Init Spark & Glue Wrappers
# ==========================================
sc = SparkContext.getOrCreate() # ใช้ getOrCreate ป้องกัน error เวลารันซ้ำใน notebook
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

# Mock Arguments
args = {
    'JOB_NAME': 'local_notebook_test',
    'MAX_CONCURRENT_THREADS': '3' # ลองปรับเลขนี้เล่นดูครับ
}
job.init(args['JOB_NAME'], args)

# ==========================================
# 2. Configuration & Mock Data
# ==========================================
MAX_WORKERS = int(args['MAX_CONCURRENT_THREADS'])
S3_BUCKET_ROOT = "s3a://mock-bucket" # ใช้ s3a protocol สำหรับ local spark

def get_job_control_list():
    # จำลอง Data ชุดเดิม
    return [
        {'table_id': 1, 'target_table': 'users', 'piority': 1, 'parallel_run': True},
        {'table_id': 2, 'target_table': 'products', 'piority': 1, 'parallel_run': True},
        {'table_id': 3, 'target_table': 'orders', 'piority': 2, 'parallel_run': False}, # Sequential
        {'table_id': 4, 'target_table': 'logs_a', 'piority': 2, 'parallel_run': True},
        {'table_id': 5, 'target_table': 'logs_b', 'piority': 2, 'parallel_run': True},
        {'table_id': 6, 'target_table': 'logs_c', 'piority': 2, 'parallel_run': True},
        {'table_id': 7, 'target_table': 'history', 'piority': 3, 'parallel_run': True},
    ]

# ==========================================
# 3. Core Logic
# ==========================================
def process_table(row):
    """
    ฟังก์ชันทำงานจริง
    """
    table_name = row['target_table']
    piority = row['piority']
    
    # จำลองเวลาทำงาน (Random นิดหน่อยเพื่อให้เห็นภาพ Parallel)
    import random
    process_time = random.uniform(1.0, 3.0) 
    
    print(f"   ▶ [Start] {table_name} (P:{piority})")
    time.sleep(process_time) # Simulate Data Processing
    
    # --- จุดที่ต้องแก้เมื่อจะต่อ S3 จริง ---
    # df = spark.createDataFrame(...) 
    # df.write.parquet(...)
    
    return f"   ✅ [Done] {table_name} ({process_time:.2f}s)"

# ==========================================
# 4. Orchestration Execution
# ==========================================
print(f"--- Starting Job with {MAX_WORKERS} Max Threads ---\n")

job_list = get_job_control_list()
all_priorities = sorted(list(set(job['piority'] for job in job_list)))

for prio in all_priorities:
    print(f"\n📢 PRIORITY GROUP: {prio}")
    print("-" * 30)
    
    current_batch = [job for job in job_list if job['piority'] == prio]
    parallel_jobs = [job for job in current_batch if job.get('parallel_run') == True]
    sequential_jobs = [job for job in current_batch if job.get('parallel_run') != True]
    
    # 4.1 Run Sequential
    if sequential_jobs:
        print(f"👉 Running Sequential ({len(sequential_jobs)} jobs)...")
        for job_row in sequential_jobs:
            print(process_table(job_row))
            
    # 4.2 Run Parallel
    if parallel_jobs:
        print(f"👉 Running Parallel ({len(parallel_jobs)} jobs)...")
        with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            future_to_table = {executor.submit(process_table, job): job for job in parallel_jobs}
            
            for future in concurrent.futures.as_completed(future_to_table):
                try:
                    print(future.result())
                except Exception as exc:
                    print(f"❌ Error: {exc}")

print("\n🎉 All Jobs Completed.")
job.commit()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/23 13:50:56 WARN Utils: Your hostname, Santis-Mac-Studio.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.38 instead (on interface en0)
26/01/23 13:50:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/23 13:50:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


   -> [Mock] Job Initialized: local_notebook_test
--- Starting Job with 3 Max Threads ---


📢 PRIORITY GROUP: 1
------------------------------
👉 Running Parallel (2 jobs)...
   ▶ [Start] users (P:1)
   ▶ [Start] products (P:1)
   ✅ [Done] users (1.68s)
   ✅ [Done] products (1.76s)

📢 PRIORITY GROUP: 2
------------------------------
👉 Running Sequential (1 jobs)...
   ▶ [Start] orders (P:2)
   ✅ [Done] orders (2.13s)
👉 Running Parallel (3 jobs)...
   ▶ [Start] logs_a (P:2)
   ▶ [Start] logs_b (P:2)
   ▶ [Start] logs_c (P:2)
   ✅ [Done] logs_a (1.67s)
   ✅ [Done] logs_b (2.62s)
   ✅ [Done] logs_c (2.83s)

📢 PRIORITY GROUP: 3
------------------------------
👉 Running Parallel (1 jobs)...
   ▶ [Start] history (P:3)
   ✅ [Done] history (2.58s)

🎉 All Jobs Completed.
   -> [Mock] Job Committed
